In [1]:
from pymongo import MongoClient
import threading
import time

<hr>

In [2]:
# Параметри з'єднання з MongoDB
uri = "mongodb://localhost:27017"

client = MongoClient(uri)
db = client["lab"] 
collection = db["likes_counter"]

client

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)

<hr>

In [3]:
# Ініціалізація колекції та лічильника "likes"
def initialize_field():
    collection.delete_many({})
    collection.insert_one({"_id": 1, "likes": 0})
    collection.insert_one({"_id": 2, "likes": 0})
    collection.insert_one({"_id": 3, "likes": 0})
    collection.insert_one({"_id": 4, "likes": 0})

In [4]:
# Інкремент лічильника на '+1' через findOneAndUpdate
def increment_likes(id, write_concern_option):
    for _ in range(10000):
        collection.find_one_and_update(
            {"_id": id}, {"$inc": {"likes": 1}},
            write_concern=write_concern_option)

<hr>

In [5]:
# Запуск інкрементів з різними 'writeConcern' та нодами
def execute_task(id, write_concern, disconnect_primary=False):
    initialize_field()
    start_time = time.time()

    threads = [threading.Thread(target=increment_likes, 
                                args=(id, {"w": write_concern}, )) 
               for _ in range(10)]

    # Запускаємо всі 10 окремих потоків
    for thread in threads: thread.start()

    if disconnect_primary:
        time.sleep(20)  # Даємо процесу попрацювати деякий час
        
        # Нотифікація адміністратора для відключення Primary-ноди
        print("Manually disconnect PrimaryNode!")

    # Очікуємо завершення всіх потоків
    for thread in threads: thread.join()

    end_time = time.time()

    # Перевірка фінального значення
    final_value = client["lab"]["likes_counter"].find_one({"_id": id})["likes"]
    print(f"Task executed in: {end_time - start_time:.2f} seconds")
    print(f"Final counter value: {final_value}")

<hr>

In [6]:
if __name__ == "__main__":
    print(f"\n{'-' * 50}\nExecuting task with 'writeConcern=1'...\n{'-' * 50}")
    execute_task(id=1, write_concern=1, disconnect_primary=False)

    print(f"\n{'-' * 50}\nExecuting task with 'writeConcern=majority'...\n{'-' * 50}")
    execute_task(id=2, write_concern="majority", disconnect_primary=False)

    print(f"\n{'-' * 50}\nExecuting task with 'writeConcern=1' and 'disconnect_primary'...\n{'-' * 50}")
    execute_task(id=3, write_concern=1, disconnect_primary=True)

    print(f"\n{'-' * 50}\nExecuting task with 'writeConcern=majority' and 'disconnect_primary'...\n{'-' * 50}")
    execute_task(id=4, write_concern="majority", disconnect_primary=True)


--------------------------------------------------
Executing task with 'writeConcern=1'...
--------------------------------------------------


ServerSelectionTimeoutError: localhost:27017: [WinError 10061] No connection could be made because the target machine actively refused it (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms), Timeout: 30s, Topology Description: <TopologyDescription id: 675d8b8f1ed6c5236d3f5f82, topology_type: Unknown, servers: [<ServerDescription ('localhost', 27017) server_type: Unknown, rtt: None, error=AutoReconnect('localhost:27017: [WinError 10061] No connection could be made because the target machine actively refused it (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms)')>]>